# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [8]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [9]:
stocks = pd.read_csv('sp_500_stocks1.csv')
from secrets import IEX_CLOUD_API_TOKEN


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [10]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2389588504380,
 'week52high': 178.3,
 'week52low': 123.98,
 'week52highSplitAdjustOnly': 179.61,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': -0.06761878245323127,
 'sharesOutstanding': 15821946000,
 'float': 0,
 'avg10Volume': 54917687,
 'avg30Volume': 59759242,
 'day200MovingAvg': 145.92,
 'day50MovingAvg': 147.08,
 'employees': 164000,
 'ttmEPS': 5.88,
 'ttmDividendRate': 0.9170023129162208,
 'dividendYield': 0.006071656710032581,
 'nextDividendDate': '',
 'exDividendDate': '2023-02-10',
 'nextEarningsDate': '2023-05-04',
 'peRatio': 25.10836814134558,
 'beta': 0,
 'maxChangePercent': 57.63876378319615,
 'year5ChangePercent': 2.6008220641248165,
 'year2ChangePercent': 0.27396110551110686,
 'year1ChangePercent': -0.06761878245323127,
 'ytdChangePercent': 0.1641701077375728,
 'month6ChangePercent': -0.02636040717126853,
 'month3ChangePercent': 0.02463659763389892,
 'month1ChangePercent': -0.0209672348466432,
 'day30ChangePercent': 0.

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [11]:
data['year1ChangePercent']

-0.06761878245323127

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [12]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [13]:
#final_dataframe = pd.DataFrame(columns = my_columns)
series=[]
for symbol_string in symbol_strings:
#   print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','): 
        s= pd.Series([symbol, 
            data[symbol]['quote']['latestPrice'],
            data[symbol]['stats']['year1ChangePercent'], 
                'N/A'], 
            index = my_columns)
        series.append(s)
final_dataframe= pd.concat(series,axis=1,ignore_index = True)
final_dataframe=final_dataframe.T
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,103.295,-0.206788,N/A
1,AOS,66.045,0.016198,N/A
2,ABT,98.04,-0.124384,N/A
3,ABBV,152.33,0.075986,N/A
4,ABMD,381.02,0,N/A
...,...,...,...,...
499,YUM,125.59,0.10799,N/A
500,ZBRA,288.31,-0.215072,N/A
501,ZBH,125.02,0.05673,N/A
502,ZION,31.235,-0.224325,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [14]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LW,97.16,0.954709,N/A
1,MPC,124.49,0.759542,N/A
2,VLO,128.445,0.693774,N/A
3,URI,415.78,0.520137,N/A
4,KSU,293.59,0.506512,N/A
5,PWR,157.15,0.497507,N/A
6,LVS,56.22,0.490196,N/A
7,ALB,222.89,0.457886,N/A
8,GWW,671.48,0.455475,N/A
9,SLB,49.275,0.449594,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [15]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [16]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\Anant Raj\AppData\Local\Temp\ipykernel_9784\2310163137.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LW,97.16,0.954709,2018
1,MPC,124.49,0.759542,1575
2,VLO,128.445,0.693774,1526
3,URI,415.78,0.520137,471
4,KSU,293.59,0.506512,667
5,PWR,157.15,0.497507,1247
6,LVS,56.22,0.490196,3487
7,ALB,222.89,0.457886,879
8,GWW,671.48,0.455475,292
9,SLB,49.275,0.449594,3979


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [17]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

# hqm_dataframe = pd.DataFrame(columns = hqm_columns)
lists=[]
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        s= pd.Series([symbol, 
        data[symbol]['quote']['latestPrice'],
        'N/A',
        data[symbol]['stats']['year1ChangePercent'],
        'N/A',
        data[symbol]['stats']['month6ChangePercent'],
        'N/A',
        data[symbol]['stats']['month3ChangePercent'],
        'N/A',
        data[symbol]['stats']['month1ChangePercent'],
        'N/A',
        'N/A'],
        index = hqm_columns)
        lists.append(s)
            
hqm_dataframe= pd.concat(lists,axis=1,ignore_index = True)
hqm_dataframe=hqm_dataframe.T
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,103.25,N/A,-0.206788,N/A,-0.062448,N/A,-0.112268,N/A,-0.040487,N/A,N/A
1,AOS,66.045,N/A,0.016198,N/A,0.223891,N/A,0.13399,N/A,-0.013324,N/A,N/A
2,ABT,98.01,N/A,-0.124384,N/A,0.028468,N/A,-0.0293,N/A,-0.065659,N/A,N/A
3,ABBV,152.32,N/A,0.075986,N/A,0.167791,N/A,-0.037373,N/A,0.074793,N/A,N/A
4,ABMD,381.02,N/A,0,N/A,0,N/A,0,N/A,0,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,125.59,N/A,0.10799,N/A,0.155439,N/A,-0.000077,N/A,0.020943,N/A,N/A
500,ZBRA,288.31,N/A,-0.215072,N/A,0.039819,N/A,0.125846,N/A,-0.064569,N/A,N/A
501,ZBH,125.02,N/A,0.05673,N/A,0.200552,N/A,0.028376,N/A,-0.024952,N/A,N/A
502,ZION,31.185,N/A,-0.224325,N/A,-0.091203,N/A,-0.050415,N/A,-0.088112,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [18]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-pa

C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-pa

C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-pa

C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-pa

0      0.178571
1      0.571429
2      0.285714
3      0.698413
4      0.531746
         ...   
499    0.767857
500    0.166667
501     0.64881
502    0.156746
503    0.281746
Name: One-Year Return Percentile, Length: 504, dtype: object
0       0.22619
1      0.825397
2      0.428571
3      0.736111
4      0.355159
         ...   
499    0.714286
500    0.456349
501    0.789683
502    0.166667
503    0.587302
Name: Six-Month Return Percentile, Length: 504, dtype: object
0      0.113095
1      0.890873
2      0.388889
3      0.355159
4      0.539683
         ...   
499    0.527778
500    0.888889
501    0.646825
502    0.329365
503    0.819444
Name: Three-Month Return Percentile, Length: 504, dtype: object
0      0.382937
1      0.585317
2      0.248016
3      0.954365
4      0.693452
         ...   
499    0.815476
500    0.253968
501    0.486111
502    0.144841
503    0.799603
Name: One-Month Return Percentile, Length: 504, dtype: object


C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
C:\Users\Anant Raj\anaconda3\lib\site-pa

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,103.25,N/A,-0.206788,0.178571,-0.062448,0.22619,-0.112268,0.113095,-0.040487,0.382937,N/A
1,AOS,66.045,N/A,0.016198,0.571429,0.223891,0.825397,0.13399,0.890873,-0.013324,0.585317,N/A
2,ABT,98.01,N/A,-0.124384,0.285714,0.028468,0.428571,-0.0293,0.388889,-0.065659,0.248016,N/A
3,ABBV,152.32,N/A,0.075986,0.698413,0.167791,0.736111,-0.037373,0.355159,0.074793,0.954365,N/A
4,ABMD,381.02,N/A,0,0.531746,0,0.355159,0,0.539683,0,0.693452,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,125.59,N/A,0.10799,0.767857,0.155439,0.714286,-0.000077,0.527778,0.020943,0.815476,N/A
500,ZBRA,288.31,N/A,-0.215072,0.166667,0.039819,0.456349,0.125846,0.888889,-0.064569,0.253968,N/A
501,ZBH,125.02,N/A,0.05673,0.64881,0.200552,0.789683,0.028376,0.646825,-0.024952,0.486111,N/A
502,ZION,31.185,N/A,-0.224325,0.156746,-0.091203,0.166667,-0.050415,0.329365,-0.088112,0.144841,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [19]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [20]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:100]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [21]:
portfolio_size=input()

10000000


In [22]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\Anant Raj\AppData\Local\Temp\ipykernel_9784\4206121469.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,103.25,968,-0.206788,0.178571,-0.062448,0.22619,-0.112268,0.113095,-0.040487,0.382937,0.225198
1,AOS,66.045,1514,0.016198,0.571429,0.223891,0.825397,0.13399,0.890873,-0.013324,0.585317,0.718254
2,ABT,98.01,1020,-0.124384,0.285714,0.028468,0.428571,-0.0293,0.388889,-0.065659,0.248016,0.337798
3,ABBV,152.32,656,0.075986,0.698413,0.167791,0.736111,-0.037373,0.355159,0.074793,0.954365,0.686012
4,ABMD,381.02,262,0,0.531746,0,0.355159,0,0.539683,0,0.693452,0.53001
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CE,105.93,944,-0.118654,0.291667,0.180623,0.751984,0.148046,0.912698,-0.001488,0.674603,0.657738
96,CNC,64.565,1548,-0.197327,0.200397,-0.232982,0.03373,-0.194895,0.019841,-0.02732,0.468254,0.180556
97,CNP,28.2,3546,0.003437,0.543651,-0.094609,0.150794,-0.067638,0.248016,-0.020699,0.52381,0.366567
98,CDAY,68.32,1463,0.115857,0.775794,0.281632,0.892857,0.067843,0.771825,-0.022966,0.5,0.735119


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [23]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [26]:
writer.save()